
## Discriminated Unions

Dealing with entities that belong to a well-defined group of cases.


One of the key aspects of F\# (and other functional languages) is that it provides a specific syntax for modeling those input and output features that belong to a well-defined collection. These are called discriminated unions (_discriminated unions_).

The type for our food `Vending Machine` would be:

In [5]:
type FoodProduct =
    | Chips
    | Chocolate
    | Candy 

while for electronics we could have:

In [2]:
type Electronics = 
    | Phones
    | Speakers
    | Headphones

> 🔔 It is customary to use `PascalCase` for types (ie `FoodProduct`).

> ❗️ Case types in a discriminated union must begin with a capital letter (`Phones`, `Chips`, etc.).

In the above expressions we are defining a type with a name (`FoodProduct`, `Electronics`) that can have multiple case types. It is important to emphasize that the cases of each type of union discriminated are _disjoint_, that is, they cannot be accessed at the same time. For example:

In [4]:
let d = FoodProduct.Chips
let ch = FoodProduct.Chocolate
let s = Electronics.Speakers

The language uses the period `.` to represent a case of discriminated union. The value `s` represents, of course, a speaker, and since it is immutable, there is no possible way it could be a phone or a headset.

So how can we define functions with discriminated unions? Let's write a `price`` function that takes an input of type `Electronics` and returns the price:

In [3]:
let price electronic = 
    match electronic with 
    | Phones -> 435
    | Speakers -> 29
    | Headphones -> 122

printfn "The price of speaker is: %A $" (price Electronics.Speakers)    

The price of speaker is: 29 $


The way one can disaggregate the different cases of an input that is a discriminated union is through _pattern matching_, represented by the `match ... with` construct and then **all** the possible cases of the union. The syntax is quite simple: for each discriminated union case label (after the `|` sign in the construct) appears the `->` arrow indicating the value (an `int`) that the function returns in that case.

Pattern matching is _exhaustive_: the match must contain _all_ possible instances of the discriminated union. If one is missing any cases, the compiler will let us know with a wavy underline in the match.

Another very important aspect of pattern matching is that it is evaluated _in the order it is written_.

But what if we want to write a `priceFood` function and assign a price of 1.5 to all items except chocolates, which are priced at 2.35? The language introduces the _wildcard_ symbol that matches _any_ input into the pattern matching construct. The wildcard is represented by the symbol `_` (underscore):

In [7]:
let priceFood food = 
    match food with 
    | Chocolate -> 2.35
    | _ -> 1.5 

printfn "Chocolate: %A $" (priceFood FoodProduct.Chocolate)        
printfn "Chips: %A $" (priceFood FoodProduct.Chips)        
printfn "Candy: %A $" (priceFood FoodProduct.Candy)        

Chocolate: 2.35 $
Chips: 1.5 $
Candy: 1.5 $


Here we see the interaction between pattern matching order evaluation and the wildcard. We are returning specific values ​​for _some cases_ (Chocolate), and assigning a common value for _all other cases_. When `priceFood` receives a "value" of food, it is first compared to the case of "Chocolate". If it is not chocolate, the wild pattern captures it. Consider the following example:

In [6]:
let priceSale food = 
    match food with 
    | _ -> 1.5 
    | Chocolate -> 2.35

printfn "Chocolate: %A $" (priceSale FoodProduct.Chocolate)        
printfn "Chips: %A $" (priceSale FoodProduct.Chips)        
printfn "Candy: %A $" (priceSale FoodProduct.Candy)        

Chocolate: 1.5 $
Chips: 1.5 $
Candy: 1.5 $


Well, everything is "on sale" at 1.5! This is because the wildcard captures _any input_ and since it is the first _case_, any food will have that price.

Here again, the behind-the-scenes compiler comes to our rescue. It will let you know that some of the pattern matching rules will not be hit:

<img src="../img/rule_will_never_match.png" alt="This will save you from bankruptcy" width="400"/>

### Combining Discrete Unions and Basic Types

In many cases, the discriminated union construct is not general enough, so it can be combined with basic types. Let's say we want to identify the brand of each of our items in vending machines. Since it can accommodate multiple brands of products, we decided that we represent the brand by a string. We can expand our `FoodProduct` discriminated union type as:

In [10]:
type BrandedFood =
    | Chips of string 
    | Chocolate of string 
    | Candy of string 

Each of the instances of type `BrandedFood` now has a _value_ of type `string`. The discriminated union uses the `of` keyword to associate each case with each value type. One can define identifiers for this composite type as:

In [11]:
let belovedChocolate = BrandedFood.Chocolate "Wonka"
let healthyChips = BrandedFood.Chips "NotALeis"
let sourCandy = BrandedFood.Candy "TearDrops"

Of course, we can have as many candy's brands (or any other product) as we want:

In [12]:
let sweetCandy = BrandedFood.Candy "SweetTreats"
let spicyCandy = BrandedFood.Candy "SpicyGummies"
let hotChips = BrandedFood.Chips "JalapeñoPowers"

In the brands example, we chose to combine all instances with the same basic type, `string`. But then again, one can mix and match. For example, a chocolate can come in different presentations:

In [13]:
type ChocolatePresentation =
| Bar of float  // a chocolate Bar of a given weight
| Box of int // a package with a number of chocolate pieces    

Or, if we want to model the change of money that the vending machine returns to the customer, we can define a case in which an actual amount is returned and another that represents the fact that the customer has just put the exact money in the machine:

In [15]:
type Change =
| Amount of float
| NoChange

Pattern matching against these composite discriminated unions is again quite simple. To get the brand of a `BrandedFood` product, we can define the `brand` function:

In [12]:
let brand product = 
    match product with
    | Chips p -> p 
    | Chocolate p -> p 
    | Candy p -> p  

In [13]:
printfn "Brand of belovedChocolate: %s" (brand belovedChocolate)

Brand of belovedChocolate: Wonka


In this function, each case has an associated value represented by the identifier `p`, which is _unwrapped_ from the discriminated union and returned.

Another example:

In [17]:
let changeValue change =
    match change with 
    | Amount money -> money
    | NoChange -> 0 

let c = Amount 3 

printfn "You are receiving %A $ as change" (changeValue c)

let exactPayment = NoChange 

printfn "You are receiving no change: %A" (changeValue exactPayment)


You are receiving 3.0 $ as change
You are receiving no change: 0.0


> Note how a discriminated union that mixes basic types seems to present a kind of divergence in the code. The `Change` type combines a wrapped `float` type (with `Amount`) and a pure `NoChange` union type. Since functions must have specific input and output types, any function that receives a `Change` input has to return a defined output. There are usually two possibilities. The first is one for the function is to _flatten_ the inputs (as in the case of `changeValue`), where the float value of the money the user receives is obtained. The second corresponds to a function that promotes input types to another: for example, the trivial case would be a function that prints the amount of money the customer receives, `printChange: Change -> ()`, that is, receives a 'Change' and returns 'unit'.

> This is one of the key ideas behind functional programming, being able to connect different types through functions.

### Protection of inputs with unique discriminated unions

The last, but not least, use case of discriminated unions corresponds to those that only have one term. Let's say we need to describe the models of the different items in our electronics vending machine. One way to do this is by using a _single discriminated union_:

In [35]:
type Model =
    | Model of string

So, we can define different article models:

In [36]:
let yPhone = Model "Xtreme 3S"
let miniSpeakers = Model "Louder Pro"

Since the discriminated union only has one case, there is a short way to unwrap the value it contains:

In [37]:
let (Model yPhoneModel) = yPhone
printfn "The model of the yPhone is %A" yPhoneModel

The model of the yPhone is "Xtreme 3S"


There seems to be a lot of pomp going on in these types, why not just use a simpler 'string' instead? The answer is, again, related to functions. Let's define the `printModel` function as:

In [44]:
let printModel (model: Model) =
    let (Model value) = model 
    printfn "The model is %A" value 

In [45]:
printModel yPhone
printModel miniSpeakers

The model is "Xtreme 3S"
The model is "Louder Pro"


The signature of the `printModel` function is `Model -> ()`, which means that it receives a model value and returns the unit. Since the input is a 'Model', there is no possible way we can pass a plain string to it. In this way, one protects the input of the function in such a way that it has to be of the exact type that we decide on, and nothing else.

It should be noted that this is not a validation, i.e. you can still mix the meaning of 'Model' by constructing it with any string, the only piece of code being protected by the only discriminated union type is the input type in `printModel`. So this type does not replace validation, but it does give you cleaner code.

### Summary

The type of union discriminated in both variants (single and multiple) is a key aspect of functional programming in F\#. The ability to describe well-predefined collections with a single type simplifies code and makes it robust. The exhaustive pattern matching specification (overseen by the F\# compiler behind the scenes) ensures that the programmer deals with all instances of the discriminated union and that not a single instance is left with potential holes in the code.

The discriminated union type is the way F\# represents a _sum type_, as such types are called in algebraic type theory.